# Darknet Configuration

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 14.01 MiB | 16.01 MiB/s, done.
Resolving deltas: 100% (10339/10339), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

#Helper function for displaying image.

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

#Connecting google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/yolov4

Mounted at /content/gdrive
backup					 Own_Images
generate_test.py			 Realtime_Handgun_Detection.ipynb
generate_train.py			 Results
Handgun_Detection_YOLOv4_Testing.ipynb	 test.zip
Handgun_Detection_YOLOv4_Training.ipynb  training_loss_chart.jpg
obj.data				 yolov4-obj.cfg
obj.names				 yolov4-obj-testing.cfg
obj.zip


#Uploading files to Colab Virtual Machine necessary for training.

In [ ]:
# copy over both datasets into the root directory of the Colab VM
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test_dataset.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test_dataset.zip -d data/

In [ ]:
# download cfg to google drive and change its name
#!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-obj.cfg

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

9k.tree     giraffe.jpg		      labels		person.jpg  voc.names
coco9k.map  goal.txt		      obj		scream.jpg
coco.names  horses.jpg		      obj.data		test
dog.jpg     imagenet.labels.list      obj.names		test.txt
eagle.jpg   imagenet.shortnames.list  openimages.names	train.txt


#Getting the pre-trained Yolov4 weight file for training.

In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

#Converting the annotated image's text files to one class. (15 to 0)

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
15 0.246875 0.674118 0.231250 0.632941

['15', '0.246875', '0.674118', '0.231250', '0.632941']
1299 data/obj/armas_281_.txt
0 0.246875 0.674118 0.231250 0.632941
15 0.747321 0.413462 0.144643 0.151099

['15', '0.747321', '0.413462', '0.144643', '0.151099']
1300 data/obj/armas_678_.txt
0 0.747321 0.413462 0.144643 0.151099
15 0.754444 0.431579 0.140000 0.289474

['15', '0.754444', '0.431579', '0.140000', '0.289474']
1301 data/obj/armas_36_.txt
0 0.754444 0.431579 0.140000 0.289474
15 0.518077 0.436190 0.097692 0.194166

['15', '0.518077', '0.436190', '0.097692', '0.194166']
1302 data/obj/armas_35_.txt
0 0.518077 0.436190 0.097692 0.194166
15 0.603746 0.175000 0.135447 0.135185

['15', '0.603746', '0.175000', '0.135447', '0.135185']
1303 data/obj/armas_1889_.txt
0 0.603746 0.175000 0.135447 0.135185
15 0.500000 0.121528 0.516000 0.229167

['15', '0.500000', '0.121528', '0.516000', '0.229167']
1304 data/obj/armas_1258_.txt
0 0.500000 0.12

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/test/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.378077 0.518391 0.450000 0.448276

['15', '0.378077', '0.518391', '0.450000', '0.448276']
0 data/test/armas_2878_.txt
0 0.378077 0.518391 0.450000 0.448276
15 0.503125 0.491667 0.931250 0.916667

['15', '0.503125', '0.491667', '0.931250', '0.916667']
1 data/test/armas_2291_.txt
0 0.503125 0.491667 0.931250 0.916667
15 0.488152 0.563333 0.919431 0.733333

['15', '0.488152', '0.563333', '0.919431', '0.733333']
2 data/test/armas_2139_.txt
0 0.488152 0.563333 0.919431 0.733333
15 0.473214 0.330357 0.642857 0.553571

['15', '0.473214', '0.330357', '0.642857', '0.553571']
3 data/test/armas_2911_.txt
0 0.473214 0.330357 0.642857 0.553571
15 0.503125 0.500000 0.981250 0.950000

['15', '0.503125', '0.500000', '0.981250', '0.950000']
4 data/test/armas_2731_.txt
0 0.503125 0.500000 0.981250 0.950000
15 0.496875 0.487500 0.918750 0.958333

['15', '0.496875', '0.487500', '0.918750', '0.958333']
5 data/test/armas_2383_.txt
0 0.496875 0.487500 0.918750 0.958333
15 0.500000 0.487500 0.950000 0.92

#Command to start the training.

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

# kick off training from where it last saved
#!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

In [ ]:
imShow('chart.png')
!cp chart.png /mydrive/yolov4/chart.png

#Getting the Mean Average Precision of the trained model.
To get the most accurate model for testing.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_final.weights

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv 

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_5000.weights

In [ ]:
# !./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_5000.weights -iou_thresh 0.75

#Testing the model using the test data set.
Extracting the output into txt and json file.

In [ ]:
# !./darknet detector test data/obj.data /mydrive/yolov4/yolov4-obj-testing.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show -ext_output < data/test.txt > result_test.txt

In [ ]:
# !./darknet detector test data/obj.data /mydrive/yolov4/yolov4-obj-testing.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -ext_output -dont_show -out result_test.json < data/test.txt

#Setting the custom cfg for testing. 

In [ ]:
# %cd cfg
# !sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
# !sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
# !cp yolov4-obj.cfg /mydrive/yolov4/yolov4-obj-testing.cfg
# %cd ..

#Command for testing the model. (Images)

In [ ]:
!./darknet detector test data/obj.data /mydrive/yolov4/yolov4-obj-testing.cfg /mydrive/yolov4/backup/yolov4-obj_5000.weights /mydrive/yolov4/Own_Images/armas2.jpg -thresh 0.75
imShow('predictions.jpg')
!cp predictions.jpg /mydrive/yolov4/Results/predictions.jpg

#Command for testing the model. (Videos)

In [ ]:
!./darknet detector demo data/obj.data /mydrive/yolov4/yolov4-obj-testing.cfg /mydrive/yolov4/backup/yolov4-obj_5000.weights -dont_show /mydrive/yolov4/Own_Images/video9.mp4 -i 0 -out_filename results.mp4 -thresh 0.75
!cp results.mp4 /mydrive/yolov4/Results/

#Helper function for displaying videos

In [ ]:
import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [ ]:
# convert resulting video from avi to mp4 file format
import os
path_video = os.path.join("","results.mp4")
%cd /content/darknet
!ffmpeg -y -loglevel panic -i results.mp4 output.mp4

# output object tracking video
path_output = os.path.join("","output.mp4")
show_video(path_output, width=960)